In [1]:
from dask_gateway import Gateway, GatewayCluster

In [2]:
gateway = Gateway(
    address="http://submit.mit.edu:6820",
    proxy_address="http://submit.mit.edu:6821",
)

In [3]:
gateway.list_clusters()

[ClusterReport<name=310ce20ba9d844098b599230be32af9b, status=RUNNING>,
 ClusterReport<name=5b4eb8c3e3374bddb68b291c85acbdd0, status=RUNNING>]

In [4]:
options = gateway.cluster_options()

In [5]:
options #If ipywidgets is installed, you can change the cluster options below (type you conda environment)

In [6]:
# Iterate over the available options and print their details
for option_name, option in options.items():
    try:
        print(f"Option: {option_name} - Value: {option}")
    except AttributeError as e:
        print(f"Failed to access option '{option_name}':", e)

Option: worker_cores - Value: 1
Option: worker_memory - Value: 1.0
Option: environment - Value: base


In [7]:
# Optionally, set specific options if needed
options['worker_cores'] = 2  # Example: setting worker cores to 2
options['worker_memory'] = 4.0  # Example: setting worker memory to 4.0 GB
options['environment'] = 'AF' #put the conda environment name that you want here

In [8]:
from dask_gateway import Gateway, GatewayClusterError
# Try to create a new cluster with the specified options
try:
    cluster = gateway.new_cluster(options)
    print("Cluster created successfully")
except GatewayClusterError as e:
    cluster_name = None
    if "failed to start" in str(e):
        cluster_name = e.args[0].split("'")[1]
    print(f"Failed to create cluster: {e}")

    if cluster_name:
        # Attempt to fetch the logs for the failed cluster
        try:
            cluster = gateway.connect(cluster_name)
            logs = cluster.get_logs()
            print("Cluster logs:")
            print(logs)
        except Exception as log_e:
            print(f"Failed to fetch logs for cluster '{cluster_name}': {log_e}")
    else:
        print("Cluster name could not be extracted for log retrieval.")

Cluster created successfully


In [9]:
cluster #You can scale the cluster manually below or you can set it to use adaptive scaling

In [ ]:
cluster.scale()

In [15]:
client = cluster.get_client()

In [16]:
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: http://submit.mit.edu:6820/clusters/fed2e6f2abcd49289acc4b2fde86dacd/status,


In [17]:
import numpy as np
import dask.array as da

In [18]:
x = da.random.random((100, 100), chunks=(10, 10))
y = da.random.random((100, 100), chunks=(10, 10))

# Perform a matrix multiplication
z = da.dot(x, y)

# Compute the result (this will trigger the distributed computation)
result = z.compute()

/work/submit/freerc/miniforge3/envs/AF/lib/python3.9/site-packages/dask/array/routines.py:326: PerformanceWarning: Increasing number of chunks by factor of 10
  intermediate = blockwise(


In [19]:
result

array([[27.34846011, 27.67445794, 27.97154035, ..., 25.88894165,
        26.94553653, 25.3116628 ],
       [25.2026391 , 27.35171109, 25.90871407, ..., 25.51597468,
        27.45140399, 24.12873174],
       [23.94169729, 25.63426158, 23.26259635, ..., 23.75644834,
        25.73457245, 22.17038899],
       ...,
       [23.1896605 , 24.59618487, 22.3355991 , ..., 22.66537874,
        24.69327374, 23.15148908],
       [25.30974692, 26.84353189, 25.04249372, ..., 25.15469876,
        27.19384263, 23.23868815],
       [25.94445709, 28.63267463, 28.33337995, ..., 25.56051863,
        29.54683178, 25.22011342]])